In [11]:
import os
import numpy as np
import cv2
import torch
import torch.optim as optim
from torchvision.models.mobilenet import mobilenet_v2
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import StepLR
from torch.nn import CrossEntropyLoss
import pandas as pd
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn import preprocessing

In [12]:
class DatasetSpec(Dataset):
    
    def __init__(self, data, root_path, transform=None):
        self.data = data
        self.root_path = root_path
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        image_path = os.path.join(self.root_path, self.data.loc[index, 'id'])
        image = cv2.imread(image_path)
        label = self.data.loc[index, 'answer']
        
        if self.transform is not None:
            aug = self.transform(image=image)
            image = aug['image']
        
        return image, label

In [13]:
def train(model, device, train_loader, optimizer, epoch):
    log_interval = 10
    loss_func = CrossEntropyLoss()
    model.train()
    passed = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        #data = data.repeat(1, 1, 1, 1)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()
        passed += len(data)
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, passed, len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [14]:
def valid(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    loss_func = CrossEntropyLoss()
    with torch.no_grad():
        for data, target in test_loader:
            #data = data.repeat(1, 1, 1, 1)
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_func(output, target)
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [15]:
def main():
    batch_size = 100
    learning_rate = 1.0
    reduce_lr_gamma = 0.7
    epochs = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('Device: {} Epochs: {} Batch size: {}'.format(device, epochs, batch_size))

    kwargs = {'batch_size': batch_size}
    if torch.cuda.is_available():
        kwargs.update({'num_workers': 2, 'pin_memory': True})

    transform = A.Compose(
        [
        A.Resize(128, 128),
        A.Normalize(mean=(0.6166, 0.4851, 0.5957,), std=(0.3961, 0.4449, 0.3656,)),
        ToTensorV2(),
        ], p=1.0)
    
    full_df = pd.read_csv('../input/mlab4/train.csv')
    test_df = pd.read_csv('../input/mlab4/test_png.csv')
    
    le = preprocessing.LabelEncoder()
    full_df['answer'] = le.fit_transform(full_df['answer'])
    
    full_dataset = DatasetSpec(full_df, '../input/mlab4/train', transform=transform)
    train_size = int(0.8 * len(full_dataset))
    valid_size = len(full_dataset) - train_size
    train_dataset, valid_dataset = torch.utils.data.random_split(full_dataset, [train_size, valid_size])
    
    test_dataset = DatasetSpec(test_df, '../input/mlab4/test', transform=transform)
    print('Length train: {} Length valid: {} Length test: {}'.format(len(train_dataset), len(valid_dataset), len(test_dataset)))
    
    train_loader = DataLoader(train_dataset, shuffle=True, **kwargs)
    valid_loader = DataLoader(valid_dataset, shuffle=True, **kwargs)
    test_loader = DataLoader(test_dataset, shuffle=False, **kwargs)
    print('Number of train batches: {} Number of valid batches: {} Number of test batches: {}'.format(len(train_loader), len(valid_loader), len(test_loader)))
    
    model = mobilenet_v2(pretrained=True)
    model.classifier[1] = torch.nn.Linear(in_features=model.classifier[1].in_features, out_features=10)
    model.to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=learning_rate)

    scheduler = StepLR(optimizer, step_size=1, gamma=reduce_lr_gamma)
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        valid(model, device, valid_loader)
        scheduler.step()
        
    torch.save(model.state_dict(), "trained_model.pt")

    # Final prediction
    submission = pd.read_csv('../input/mlab4/test_wav.csv')
    predictions = []
    for data, target in test_loader:
        data = data.to(device)
        output = model(data)
        predictions += output.argmax(dim=1, keepdim=True).tolist()
    submission['answer'] = le.inverse_transform(predictions)
    submission.to_csv('submission.csv', index=False)
    print('Submission saved in: {}'.format('submission.csv'))


if __name__ == '__main__':
    main()